In [3]:
import control as co
import numpy as np
import sympy as sp 
from sympy.abc import omega, zeta

In [48]:
a, ao, b, r1, s, s0, s1, t0 = sp.symbols('a,a_o,b,r_1, s, s_0, s_1, t_0')
B = b
B_minus = b
A = s*(s + a)
R_prime = s + r1
S = s0*s + s1

Ao = s + ao
Am = s**2 + 2*zeta*omega*s + omega**2

## Diophantine Equation

In [66]:
dio_LHS = sp.collect(sp.expand(A*R_prime + B_minus*S), s)
dio_LHS

b*s_1 + s**3 + s**2*(a + r_1) + s*(a*r_1 + b*s_0)

In [67]:
dio_RHS = sp.collect(sp.expand(Ao*Am), s)
dio_RHS

a_o*omega**2 + s**3 + s**2*(a_o + 2*omega*zeta) + s*(2*a_o*omega*zeta + omega**2)

In [68]:
dio_poly = sp.Poly(dio_LHS - dio_RHS, s)
dio_poly

Poly((a - a_o - 2*omega*zeta + r_1)*s**2 + (a*r_1 - 2*a_o*omega*zeta + b*s_0 - omega**2)*s - a_o*omega**2 + b*s_1, s, domain='ZZ[a,b,a_o,omega,r_1,s_0,s_1,zeta]')

In [69]:
r1_sym = sp.solve(dio_poly.coeffs()[0], r1)
r1_sym[0]

-a + a_o + 2*omega*zeta

In [70]:
s0_sym = sp.solve(dio_poly.coeffs()[1], s0)
s0_sym = sp.simplify(sp.expand(s0_sym[0].subs(r1, r1_sym[0])))
s0_sym

(a**2 - a*a_o - 2*a*omega*zeta + 2*a_o*omega*zeta + omega**2)/b

In [71]:
s1_sym = sp.solve(dio_poly.coeffs()[2], s1)
s1_sym[0]

a_o*omega**2/b

In [72]:
t0 = omega/b
t0

omega/b

## RLS

### ODE

$p^2 y(t) + apy(t) = bu(t)$

In [76]:
uf, yf = sp.symbols('u_f, y_f')

In [77]:
Hf = 1/(Am)
Hf

1/(omega**2 + 2*omega*s*zeta + s**2)